In [13]:
import plotly
import numpy as npy
import pandas as pd

In [14]:
path = "dp_grasp_logs"
# rename all files that start with dpcfg to start with cfg
import os
for filename in os.listdir(path):
    if filename.startswith("cfgg"):
        os.rename(path + "/" + filename, path + "/cfg" + filename[4:])


In [15]:
import re
from collections import defaultdict

def parse_log_files(directory):
    # Updated regex pattern to match more complex cfg and obj formats
    pattern = re.compile(
        r"cfg-(?P<cfg>[\w\-]+)_obj-(?P<obj>[\w\-]+)_(?P<timestamp>\d+\.\d+)(_fail[-_](?P<failreason>[\w\-]+))?.csv", re.IGNORECASE
    )
    
    parsed_files = []
    
    # List all files in the given directory
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            # Extract the captured groups
            file_info = {
                "filename": f"{directory}/{filename}",
                "cfg": match.group("cfg"),
                "obj": match.group("obj"),
                "fail": match.group("failreason") is not None,
                "failreason": match.group("failreason") if match.group("failreason") else "",
            }
            parsed_files.append(file_info)
    
    return parsed_files

def group_files_by_object(parsed_files):
    # Create a dictionary with object types as keys and lists of corresponding files as values
    grouped_files = defaultdict(list)
    
    for file_info in parsed_files:
        obj = file_info['obj']
        grouped_files[obj].append(file_info)
    
    return dict(grouped_files)


# Example usage:
parsed_files = parse_log_files(path)
grouped_files = group_files_by_object(parsed_files)

# for file_info in parsed_files:
#     print(file_info)

# for obj, files in grouped_files.items():
#     print(f"Object: {obj}")
#     for file_info in files:
#         print(f"  {file_info}")



In [48]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict

# Function to parse the files and compute averages
def parse_and_average_files(dp_go_files, dp_go_nf_files):
    dp_go_data = []
    dp_go_nf_data = []

    # Process dp_go files
    for file in dp_go_files:
        df = pd.read_csv(file)
        if len(df) < 15:  # Skip files with less than 16 rows
            print(f"Skipping {file}: not enough rows")
            continue
        df = df.iloc[:15]  # Only keep the first 15 timesteps
        dp_go_data.append(df)

    # Process dp_go_nf files
    for file in dp_go_nf_files:
        df = pd.read_csv(file)
        if len(df) < 15:  # Skip files with less than 16 rows
            print(f"Skipping {file}: not enough rows")
            continue
        df = df.iloc[:15]  # Only keep the first 15 timesteps
        dp_go_nf_data.append(df)

    # Compute average per timestep
    if dp_go_data:
        dp_go_avg = pd.concat(dp_go_data).groupby(level=0).mean()
    else:
        dp_go_avg = None
    
    if dp_go_nf_data:
        dp_go_nf_avg = pd.concat(dp_go_nf_data).groupby(level=0).mean()
    else:
        dp_go_nf_avg = None

    return dp_go_avg, dp_go_nf_avg

def plot_data(dp_go_avg, dp_go_nf_avg, object_name):
    fig = go.Figure()

    # Plot dp_go gripper position (left y-axis)
    if dp_go_avg is not None:
        fig.add_trace(go.Scatter(x=np.arange(15)*0.25, y=dp_go_avg['robot_state/gripper_position'], 
                                 mode='lines', name='Position', yaxis="y1"))

    # Plot dp_go_nf gripper position (left y-axis)
    if dp_go_nf_avg is not None:
        fig.add_trace(go.Scatter(x=np.arange(15)*0.25, y=dp_go_nf_avg['robot_state/gripper_position'], 
                                 mode='lines', name='Position (No Force)', yaxis="y1"))
        # print(dp_go_nf_avg['robot_state/gripper_position'])
    # Plot dp_go applied force (right y-axis, dashed and smaller width)
    if dp_go_avg is not None:
        fig.add_trace(go.Scatter(x=np.arange(15)*0.25, y=dp_go_avg['robot_state/applied_force'], 
                                 mode='lines', name='Applied Force', yaxis="y2", marker=dict(size=1), 
                                 line=dict(dash='dash', width=1)))

        # Plot dp_go contact force (right y-axis, dashed and smaller width)
        fig.add_trace(go.Scatter(x=np.arange(15)*0.25, y=dp_go_avg['robot_state/contact_force'], 
                                 mode='lines', name='Contact Force', yaxis="y2", 
                                 line=dict(dash='dash', width=1)))

    # Set up the layout
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis=dict(
            title="Gripper Position (mm)",
            side="left",
            tickmode='linear',
            range=[0, 106],
            # range=[0, 66],
            dtick=10,  # Y-axis interval of 10mm
        ),
        yaxis2=dict(
            title="Force (N)",
            overlaying="y",
            side="right",
            tickmode='linear',
            dtick=0.2,  # Y-axis interval of 0.2N
        ),
        legend=dict(
            x=0,              # Keep legend aligned to the left
            y=1.11,            # Elevate the legend higher to avoid overlap
            orientation="h",  # Horizontal layout for the legend
            title=f"{object_name}",
            font=dict(
                size=9       # Smaller font to fit in one row
            ),
            itemsizing='trace',  # Keep legend item size constant
        ),
        width=525,  # Plot width
        height=370,  # Plot height
        margin=dict(l=50,r=50,b=0,t=1),
    )

    fig.show()
    save_pth = f"dp_plots/{object_name}.png"
    # fig.write_image(save_pth)



# Using the previously created grouped_files
def generate_plots_for_grouped_files(grouped_files):
    for obj, files in grouped_files.items():
        # Separate dp_go and dp_go_nf files
        dp_go_files = [f['filename'] for f in files if 'dp_go' == f['cfg'] and "nograsp" not in f['failreason']]
        dp_go_nf_files = [f['filename'] for f in files if 'dp_go_nf' == f['cfg'] and "nograsp" not in f['failreason']]
        print(dp_go_nf_files)
        
        # Parse and average the data
        dp_go_avg, dp_go_nf_avg = parse_and_average_files(dp_go_files, dp_go_nf_files)
        
        # Generate the plot
        plot_data(dp_go_avg, dp_go_nf_avg, obj)

generate_plots_for_grouped_files(grouped_files)


['dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777264.5447490_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777264.5447497_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777264.5447498_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777264.5447499_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777387.944049.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777407.0607724_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777474.694057_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777509.8597229.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-blackberry_1728777530.93879.csv']
Skipping dp_grasp_logs/cfg-dp_go_obj-blackberry_1728772117.144111_FAIL-crush.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776796.46661.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776825.6352503.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776866.7319517.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776909.9099495.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776933.3016274.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776983.5486338.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728777000.9165933.csv']
Skipping dp_grasp_logs/cfg-dp_go_nf_obj-egg_1728776825.6352503.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728777919.606796.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728777963.1791582.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728778052.289209.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728778071.4888294.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728778114.1269128.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728778135.238302.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728778160.7995675.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_can_1728778175.6071832.csv']
Skipping dp_grasp_logs/cfg-dp_go_obj-empty_can_1728772877.3742075.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-empty_cup_1728778757.3417761.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_cup_1728778772.9735153.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_cup_1728778783.8857565.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_cup_1728778801.036983.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_cup_1728778816.0839183.csv']
Skipping dp_grasp_logs/cfg-dp_go_obj-empty_cup_1728773947.1529994.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-empty_taco_1728777067.16367.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_taco_1728777087.3339605.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-empty_taco_1728777127.5460742.csv']
Skipping dp_grasp_logs/cfg-dp_go_obj-empty_taco_1728773092.6769967_fail-crush.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776366.2892716_FAIL-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776417.6891367_FAIL-slip.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776466.4723065.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776485.0793614.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776556.3420665.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776598.9351614.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776617.2946427.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776635.852187.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776655.7476835.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776677.0909872.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776691.9226491.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776723.9009001.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776744.8335423.csv']
Skipping dp_grasp_logs/cfg-dp_go_nf_obj-pepper_1728776635.852187.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-potato_chip_1728777657.442528.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-potato_chip_1728777693.9083185.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-potato_chip_1728777731.062843_faIl-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-potato_chip_1728777794.4455364.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-potato_chip_1728777825.0617542.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-potato_chip_1728777839.2406702.csv']


['dp_grasp_logs/cfg-dp_go_nf_obj-raspberry_1728778919.8339052_fail-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-raspberry_1728778974.922143_FAIl-crush.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-raspberry_1728779005.0830047_fail-crush.csv']
Skipping dp_grasp_logs/cfg-dp_go_obj-raspberry_1728779226.6483052.csv: not enough rows
Skipping dp_grasp_logs/cfg-dp_go_obj-raspberry_1728779453.4185297.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778217.021393.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778248.3711538.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778294.8771932.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778312.7889292.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778327.2688272.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778344.1243913.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778363.9702687.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778424.4864223.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778452.074706.csv']
Skipping dp_grasp_logs/cfg-dp_go_obj-tomato_1728773741.3249161.csv: not enough rows
Skipping dp_grasp_logs/cfg-dp_go_nf_obj-tomato_1728778424.4864223.csv: not enough rows


['dp_grasp_logs/cfg-dp_go_nf_obj-water_cup_1728778589.7921667.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-water_cup_1728778621.5921195.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-water_cup_1728778650.0853155.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-water_cup_1728778666.736021.csv', 'dp_grasp_logs/cfg-dp_go_nf_obj-water_cup_1728778713.6463194.csv']


In [37]:
# no grasp rate
go_ng = (2 + 2 +2 +1 + 2 +2 +2)
gonf_ng = (3 + 2+ 1 +2+6+2+5+4)
go_ng/(100+go_ng), gonf_ng/(100+gonf_ng)

(0.11504424778761062, 0.2)

In [57]:
# objects: blackberry, egg, empty_can, empty_cup, empty_taco, pepper, potato_chip, raspberry, tomato, water_cup
seen = ["empty_cup", "raspberry", "tomato", "water_cup"]
unseen = ["blackberry", "egg", "empty_can", "empty_taco", "pepper", "potato_chip"]
gonf_results = {
    "blackberry": 4,
    "egg": 10,
    "empty_can": 7,
    "empty_cup": 8,
    "empty_taco": 0,
    "pepper": 7,
    "potato_chip": 8,
    "raspberry": 3,
    "tomato": 7,
    "water_cup": 0,
}

go_results = {
    "blackberry": 8,
    "egg": 10,
    "empty_can": 9,
    "empty_cup": 9,
    "empty_taco": 3,
    "pepper": 8,
    "potato_chip": 10,
    "raspberry": 7,
    "tomato": 10,
    "water_cup": 8,
}

# Prepare data for plotting
seen_go = [go_results[obj] for obj in seen]
seen_go_success = "{:.0%}".format(sum(seen_go)/(len(seen_go)*10))
seen_gonf = [gonf_results[obj] for obj in seen]
seen_gonf_success = "{:.0%}".format(sum(seen_gonf)/(len(seen_gonf)*10))
unseen_go = [go_results[obj] for obj in unseen]
unseen_go_success = "{:.0%}".format(sum(unseen_go)/(len(unseen_go)*10))
unseen_gonf = [gonf_results[obj] for obj in unseen]
unseen_gonf_success = "{:.0%}".format(sum(unseen_gonf)/(len(unseen_gonf)*10))
go_success = "{:.0%}".format((sum(seen_go) + sum(unseen_go))/((len(seen_go) + len(unseen_go))*10))
gonf_success = "{:.0%}".format((sum(seen_gonf) + sum(unseen_gonf))/((len(seen_gonf) + len(unseen_gonf))*10))

all_objects = seen + unseen

# Create bar traces for "seen" objects
seen_trace_go = go.Bar(name=f'Seen ({seen_go_success})', x=seen, y=seen_go, marker_color='blue')
seen_trace_gonf = go.Bar(name=f'NF Seen ({seen_gonf_success})', x=seen, y=seen_gonf, marker_color='lightblue')

# Create bar traces for "unseen" objects
unseen_trace_go = go.Bar(name=f'Unseen ({unseen_go_success})', x=unseen, y=unseen_go, marker_color='green')
unseen_trace_gonf = go.Bar(name=f'NF Unseen ({unseen_gonf_success})', x=unseen, y=unseen_gonf, marker_color='lightgreen')

# Combine the traces and plot the figure
fig = go.Figure(data=[seen_trace_go, seen_trace_gonf, unseen_trace_go, unseen_trace_gonf])

# Update the layout
fig.update_layout(
    barmode='group',
    # title='Success Rates Comparison: GO vs GO NF',
    xaxis_title='Objects',
    yaxis_title='Success Rate (out of 10)',
    legend_title='Method',
    # xaxis={'categoryorder': 'category ascending'},
    xaxis=dict(categoryorder='array', categoryarray=all_objects),
    height=400,
    width=1200,
    legend=dict(
        x=0,              # Keep legend aligned to the left
        y=1.11,            # Elevate the legend higher to avoid overlap
        orientation="h",  # Horizontal layout for the legend
        title=f"With ({go_success}) and Without (NF: {gonf_success}) Force",
        font=dict(
            size=16       # Smaller font to fit in one row
        ),
        itemsizing='trace',  # Keep legend item size constant
    ),
    margin=dict(l=50,r=50,b=0,t=1),
)

# Show the plot
fig.show()


In [44]:
# slip vs crush
go_slip = (1 + 1 + 1 + 2 + 2)
gonf_slip = (1 + 1 + 1)
go_nfails = 18
gonf_nfails = 46
go_slip/(go_nfails+go_slip), gonf_slip/(gonf_nfails+gonf_slip)

(0.28, 0.061224489795918366)

In [43]:
go_slip/100, gonf_slip/100

(0.07, 0.03)

In [58]:
# create list of unique objects in trajectories
objects = []
for 

In [59]:
objects = "orange bottle, peeled garlic clove, stuffed animal, garlic clove, green block, tomato, red screwdriver handle, scallion stalk, small avocado, yellow ducky, water bottle, small black motor, empty paper cup, circuit board, red button, scalion stalk, orange noodle bag, yellow block, strawberry, bottle cap, small suction cup, light green chip, ziptie bag, metal lock, yellow ducky head, cardboard box, raspberry, large bearing, paper cup with water, small red green apple, paper airplane, green circuit board, plastic bottle, cherry tomato, mushroom, garlic bulb"

['blackberry',
 'egg',
 'empty_can',
 'empty_cup',
 'empty_taco',
 'pepper',
 'potato_chip',
 'raspberry',
 'tomato',
 'water_cup']